In [1]:
import pycuda
import pycuda.driver as cuda
import pycuda.autoinit

from pycuda.compiler import SourceModule

import numpy as np
import time

print(pycuda.VERSION)

"""
Matrix multiplication example
multiply (N x M) matrix and (M x N) martix
"""

(2019, 1, 1)


'\nMatrix multiplication example\nmultiply (N x M) matrix and (M x N) martix\n'

In [2]:
m = np.int32(1024)
k = np.int32(256)
n = np.int32(512)
MAT_SIZE1 = (m, k)
MAT_SIZE2 = (k, n)
RES_SIZE = (m, n)

In [3]:
# Create a tensor and copy it to gpu memory
# Randomly initialize a vector
a = np.ones(shape=MAT_SIZE1, dtype=np.float32)
a = a.astype(np.float32)
b = np.ones(shape=MAT_SIZE2, dtype=np.float32)
# Allocate memory at device
a_gpu = cuda.mem_alloc(a.nbytes)
b_gpu = cuda.mem_alloc(b.nbytes)
# Allocate same size of memory for result vector
result = np.zeros(shape=RES_SIZE, dtype=np.float32)
result_gpu = cuda.mem_alloc(result.nbytes)
# Copy the cpu vector to gpu
cuda.memcpy_htod(a_gpu, a)
cuda.memcpy_htod(b_gpu, b)

# Copy matirx information to gpu mem
m_gpu = cuda.mem_alloc(4)
k_gpu = cuda.mem_alloc(4)
n_gpu = cuda.mem_alloc(4)

cuda.memcpy_htod(m_gpu, m)
cuda.memcpy_htod(k_gpu, k)
cuda.memcpy_htod(n_gpu, n)

In [4]:
np.array(range(32)).reshape(4,8)

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23],
       [24, 25, 26, 27, 28, 29, 30, 31]])

In [5]:
np.array(range(32)).reshape(8,4)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15],
       [16, 17, 18, 19],
       [20, 21, 22, 23],
       [24, 25, 26, 27],
       [28, 29, 30, 31]])

In [6]:
# Square matrix m x k , k x n = m x n matrix
mod = SourceModule("""
    __global__ void matMul2D(float *a, float *b, float *res, int m, int k, int n)
    {
        // get row, col idx
        int rowIdx = blockDim.x * blockIdx.x + threadIdx.x;
        int colIdx = blockDim.y * blockIdx.y + threadIdx.y;
        
        // one thread per each pixels
        // I think it should be optimized
        if (rowIdx < m && colIdx < n)
        {
            int element = 0;
            for (int i=0; i<k; i++)
            {
                element += a[rowIdx*k+i] * b[i*n+colIdx];
            }
            res[rowIdx*n+colIdx] = element;
        }
    }
""")

In [7]:
# Let's compare operating time
# GPU
startTime = time.time()
func = mod.get_function("matMul2D")
func(a_gpu, b_gpu, result_gpu, m, k, n, block=(16,16,1), grid =(int(m/16),int(n/16)))
consumedTime = time.time() - startTime
print("Time for gpu operation : ", consumedTime)


Time for gpu operation :  0.0010352134704589844


In [8]:
# CPU
startTime = time.time()
result_cpu = np.dot(a, b)
consumedTime = time.time() - startTime
print("Time for cpu operation : ", consumedTime)

Time for cpu operation :  0.015708208084106445


In [9]:
# Copy the result from device to host
#result = np.zeros(shape=(4), dtype=np.float32)
cuda.memcpy_dtoh(result, result_gpu)
a_gpu.free()
b_gpu.free()
result_gpu.free()
m_gpu.free()
n_gpu.free()
k_gpu.free()

In [10]:
# Compare the results
print("Is it same? : ", (result == result_cpu).all())

Is it same? :  True


In [11]:
result_cpu

array([[256., 256., 256., ..., 256., 256., 256.],
       [256., 256., 256., ..., 256., 256., 256.],
       [256., 256., 256., ..., 256., 256., 256.],
       ...,
       [256., 256., 256., ..., 256., 256., 256.],
       [256., 256., 256., ..., 256., 256., 256.],
       [256., 256., 256., ..., 256., 256., 256.]], dtype=float32)

In [12]:
result

array([[256., 256., 256., ..., 256., 256., 256.],
       [256., 256., 256., ..., 256., 256., 256.],
       [256., 256., 256., ..., 256., 256., 256.],
       ...,
       [256., 256., 256., ..., 256., 256., 256.],
       [256., 256., 256., ..., 256., 256., 256.],
       [256., 256., 256., ..., 256., 256., 256.]], dtype=float32)